# I. Counting
- Permutations
- Combinations

# II. Probability distributions

- Probability distribution functions for discrete random variables (binomial, geometric, Poisson)
- Probability distribution for continuous random variables (uniform, normal, exponential)
- Generating simulations in python

# III. Conditional Probability
- Conditional Probability
- Bayes formula


# I. Counting 

### QUESTION (Permutations) 
How many different strings can you make with 'name'? with 'data'? with 'better'?

#### ANSWER: 
You can make 24 different strings with 'name', 12 with 'data', and 180 with 'better'

#### EXPLANATION: 
The factorial of the length of the word will yield all possible permutations of the letters. In order to count distinct permutations, one needs to account for repeated letters in the input. To do this, divide the number of all possible permutations by the number of permutations of repeated letters. We can do this with code by de-duplicating a list of all possible permutations.

In [108]:
import itertools

def make_strings(string):
    '''
    return the number of distinct strings that can be made from the characters
    inside the argument string

    use itertools.permutations

    >>> my_string = 'test'
    >>> make_strings(my_string)
    '''
    return len(set(list(itertools.permutations(string))))

In [109]:
print(make_strings('name'), make_strings("data"), make_strings('better'))

24 12 180


In [110]:
def unique_permutations(seq):
    """
    Yield only unique permutations of seq in an efficient way.

    A python implementation of Knuth's "Algorithm L", also known from the 
    std::next_permutation function of C++, and as the permutation algorithm 
    of Narayana Pandita.
    """

    # Precalculate the indices we'll be iterating over for speed
    i_indices = range(len(seq) - 1, -1, -1)
    k_indices = i_indices[1:]

    # The algorithm specifies to start with a sorted version
    seq = sorted(seq)

    while True:
        yield seq

        # Working backwards from the last-but-one index,           k
        # we find the index of the first decrease in value.  0 0 1 0 1 1 1 0
        for k in k_indices:
            if seq[k] < seq[k + 1]:
                break
        else:
            # Introducing the slightly unknown python for-else syntax:
            # else is executed only if the break statement was never reached.
            # If this is the case, seq is weakly decreasing, and we're done.
            return

        # Get item from sequence only once, for speed
        k_val = seq[k]

        # Working backwards starting with the last item,           k     i
        # find the first one greater than the one at k       0 0 1 0 1 1 1 0
        for i in i_indices:
            if k_val < seq[i]:
                break

        # Swap them in the most efficient way
        (seq[k], seq[i]) = (seq[i], seq[k])                #       k     i
                                                           # 0 0 1 1 1 1 0 0

        # Reverse the part after but not                           k
        # including k, also efficiently.                     0 0 1 1 0 0 1 1
        seq[k + 1:] = seq[-1:k:-1]

In [111]:
%%timeit
len(list(unique_permutations('better')))

227 µs ± 10.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [113]:
%%timeit
make_strings("better")

107 µs ± 4.7 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### QUESTION (Combinations)
Given a fruit bowl with 6 fruits (say for instance a pear, a banana, an apple, a pineapple, a kiwi and a mango), how many different fruit salad can you make, such that each salad contains 3 different fruits? Can you list them?

#### ANSWER: 
20

#### EXPLANATION: 
To calculate the number of combinations of r
                    items that can be selected from a set of n items, divide
                    n-factorial by r-factorial multiplied by n-minus-r factorial
                    (binomial coefficient).
                    n-choose-r = n! / r! * (n - r)!


To find combinations of three fruits from a group of 6,
                    divide 6 factorial by 3 factorial multiplied by 3-factorial.

    6! / (3! * (6 - 3)!)
    = 720 / (6 * 6)
    = 20

    ('pear', 'banana', 'apple'),
     ('pear', 'banana', 'pineapple'),
     ('pear', 'banana', 'kiwi'),
     ('pear', 'banana', 'mango'),
     ('pear', 'apple', 'pineapple'),
     ('pear', 'apple', 'kiwi'),
     ('pear', 'apple', 'mango'),
     ('pear', 'pineapple', 'kiwi'),
     ('pear', 'pineapple', 'mango'),
     ('pear', 'kiwi', 'mango'),
     ('banana', 'apple', 'pineapple'),
     ('banana', 'apple', 'kiwi'),
     ('banana', 'apple', 'mango'),
     ('banana', 'pineapple', 'kiwi'),
     ('banana', 'pineapple', 'mango'),
     ('banana', 'kiwi', 'mango'),
     ('apple', 'pineapple', 'kiwi'),
     ('apple', 'pineapple', 'mango'),
     ('apple', 'kiwi', 'mango'),
     ('pineapple', 'kiwi', 'mango')

In [125]:
 def make_fruit_salad(lst, k):
      '''
      return the list of possible combinations by taking k elements from lst

      use itertools.combinations

      >>> my_fruits = ['pear', 'banana', 'apple']
      >>> make_fruit_salad(my_fruits, 2)
      [('pear', 'banana'), ('pear', 'apple'), ('banana', 'apple')]
      '''
      return len(list(itertools.combinations(lst, k)))

## Probability Distributions
Common problems relying on discrete (Binomial, Geometric, Poisson) or continuous (Uniform, Normal, Exponential) probability distributions.


### QUESTION
You call 2 Ubers and 3 Lyfts. If the time that each takes to reach you are independent and identical distributions, what is the probability that all the Lyfts arrive first? What is the probability that all the Ubers arrive first?  


#### ANSWER: 
.01

 
#### EXPLANATION: 
3/5 * 2/4 * 1/3 = 1/10.

If the probability distributions are all independent and identical, then the chances that the first car to arrive is a Lyft is 3/5, the chances that the second car to arrive is a Lyft is 2/4, and the chances that the third car to arrive is a Lyft is 1/3. Multiply these together to find the joint probability.


### QUESTION
Consider a group of 20 people. If everyone shakes hands with everyone else, how many handshakes take place?

#### ANSWER
190 handshakes

#### EXPLANATION
This is asking how many combinations of two can be drawn
from a group of 20 individuals. Based on the formula above,
we can find the solution like so:
    20! / (2! * (20 - 2)!)
    19 * 20 / 2
    = 190


### QUESTION
 A salesman has scheduled two appointments to sell encyclopedias. His first appointment will lead to a sale with probability 0.3, and his second will lead independently to a sale with probability 0.6. Any sale made is equally likely to be either for the deluxe model, which costs $1000, or the standard model, which costs $500. Determine the probability mass function of X, the total dollar value of all sales.

#### ANSWER:

| x    | p(x)  |
|------|-------|
| 0    | 0.280 |
| 500  | 0.270 |
| 1000 | 0.315 |
| 1500 | 0.090 |
| 2000 | 0.045 |


#### QUESTION
A gambling book recommends the following “winning strategy” for the game of roulette: Bet $1 on red. If red appears (which has probability 18/38), then take the $1 profit and quit. If red does not appear and you lose this bet (which has probability 20/38 of occurring), make additional $1 bets of red on each of the next two spins of the roulette wheel and then quit. Let X denote your winnings when you quit.

(a) Find P(X > 0).

(b) Find E[X].

(c) Are you convinced that the strategy is indeed a “winning” strategy?

Answer:

| roll_1 | roll_2 | roll_3 | p_roll_1 | p_roll_2 | p_roll_3 | winnings | total_prob | prod     |           |
|--------|--------|--------|----------|----------|----------|----------|------------|----------|-----------|
| 0      | -1     | -1     | -1       | 0.526316 | 0.526316 | 0.526316 | -3         | 0.145794 | -0.437382 |
| 1      | -1     | -1     | 1        | 0.526316 | 0.526316 | 0.473684 | -1         | 0.131214 | -0.131214 |
| 2      | -1     | 1      | -1       | 0.526316 | 0.473684 | 0.526316 | -1         | 0.131214 | -0.131214 |
| 3      | -1     | 1      | 1        | 0.526316 | 0.473684 | 0.473684 | 1          | 0.118093 | 0.118093  |
| 4      | 1      | 0      | 0        | 0.473684 | 1.000000 | 1.000000 | 1          | 0.473684 | 0.473684  |

(a) p(X > 0) = 0.592 (sum of prod when winnings > 0)

(b) E[X] = -0.108 (sum of all prod)

(c) No, the expected value of winnings from each game is negative. The house
    always wins, as they say.

### QUESTION (Binomial Distribution)
The forecast says that in the next five days the chance of rain for each day is 25%. Suppose that the weather on each day does not depend on the weather on the other days. What is the probability that it will rain for at least two days in the next five days? For how many days on average will it rain in the next five days?

#### ANSWER:
Where X = probability of rain in next five days:

    P(X >= 2) = 0.367
    E[X] = 1.25

#### EXPLANATION:
    P(X >= 2) = 1 - P(X < 2)
              = 1 - (P(X = 0) + P(X = 1))
              = 1 - (0.75^5   + (5 * 0.25 * 0.75^4))

E(X) = 5 * 0.25

- Experiment to implement: simulate the number of days of rain in the next five days, knowing the 25% forecast.
- Generate a large number of experiments to obtain the ratio of experiments in which that number is at least 2 over the total number of experiments.


In [117]:
from scipy.stats import binom

def probability_rain(simulation_size=2000):
    '''
    choose the simulation_size

    returns
    -------
    probability that it will rain for at least two days in the next five days,
    knowing that the forecast says that in the next five days the chance of rain
    for each day is 25%
    '''
    chance_rain = binom.rvs(n=5, p=0.25, size=simulation_size)
    return len(chance_rain[chance_rain > 1])/float(simulation_size)

### QUESTION (Geometric distribution)
Suppose you have an unfair coin, with an 80 % chance of getting tails. What is the probability that the first head will be on the 10th trial?

#### ANSWER: 0.02684

#### EXPLANATION
Using probability mass function for geometric distribution:

We need to calculate the probability of 9 failures, multiplied by the probability
of one success.

```
    (1-p)**(k-1)*p
    (1-0.2)**(10-1)*0.2
    = 0.0268
```

#### CODE:
  - Experiment to implement: simulate the number of trials until the first success.
  - Generate a large number of experiments to obtain the ratio of experiments in which that number is at least 10 over the total number of experiments.

  If the probability of success on each trial is p, then the probability that the kth trial (out of k trials) is the first success is `geom.pmf(k) = (1-p)**(k-1)*p`


In [118]:
from scipy.stats import geom

def probability_coin(p=0.8, simulation_size=2000):
    '''
    choose the simulation_size
    p: probability of tails on a single flip of the coin

    use geom.rvs

    returns
    -------
    probability that the first head will be on the 10th trial, knowing
    that you have an unfair coin, with an p chance of getting tails.
    '''
    chance_heads = geom.rvs(1-p, size=simulation_size)
    return len(chance_heads[chance_heads == 10])/float(simulation_size)

### QUESTION (Poisson distribution)
The number of times that a person contracts a cold in a given year is a Poisson random variable with parameter λ = 5. Suppose that a new wonder drug (based on large quantities of vitamin C) has just been marketed that reduces the Poisson parameter to λ = 3 for 75 percent of the population. For the other 25 percent of the population, the drug has no appreciable effect on colds. If an individual tries the drug for a year and has 2 colds in that time, how likely is it that the drug is beneficial for him or her?

#### ANSWER
Use Bayes Theorem to find the probability that the drug works, given that an individual has experienced two colds in the past year, i.e.,
```
  P(drug_works|two_colds)

  P(A|B) = P(B|A) * P(A) / (P(B|A) * P(A) + P(B|notA) * P(notA)

  P(A) = .75

  P(notA) = 1 - prob_A = .25

  P(B|A) = poisson probability mass function of two colds when drug working
  P(x;λ)  = (e^−λ * λ^x) / x!

         Where λ is the shape parameter which indicates the average number of
         events in the given time interval.

  P(2;3) = ((~2.718) ^ -3 * 3^2 )/ 2!
         = 0.22


  P(B|notA) = poisson pmf of two colds when drug not working
  P(2;5)    = p(x;λ)= (e^−λ * λ^x) / x!
            = (~2.718) ^ -5 * 5^2 / 2!
            = 0.084
```

In [119]:
from scipy.stats import poisson

def is_drug_effective(num_colds, l_drug, l_prior):
    '''
    num_colds: number of colds the person had over the 1 year period
    l_drug: parameter of a Poisson random variable that describes the number
    of times that a person contracts a cold in a given year taking the drug.
    l_prior: parameter of a Poisson random variable that describes the number
    of times that a person contracts a cold in a given year.

    use scipy.stats.poisson.pmf
    '''
    return .75 * poisson.pmf(num_colds, one_drug) / \
            (.75 * poisson.pmf(num_colds, one_drug) + \
            (1 - .75) * poisson.pmf(num_colds, one_prior))


### QUESTION (Exponential distribution)

The number of years a radio functions is exponentially distributed with parameter λ = 1 / 8 . If Jones buys a used radio, what is the probability that it will be working after an additional 8 years?

#### ANSWER
.0459

#### EXPLANATION:
  Use probability density function for an exponential distribution to calculate radio lasting 8 years or less:
```
  f(x; λ)     = 1 − e^(−λx)
  f(8; 0.125) = 1 - (~2.718)^(-0.125*8)
              = 0.63212...
```
need to take inverse to get probability that radio lasts longer than 8 years:
```
  1 - 0.63212 = 0.368
```

in Python:
```python
  from scipy.stats import expon
  expon.sf(8, scale=8)
```

#### CODE:
- Experiment to implement: simulate the number of years the used radio will work, given the exponential distribution.
- Generate a large number of experiments to obtain the ratio of experiments in which the used radio will be working after an additional 8 years over the total number of experiments.


In [106]:
from scipy.stats import expon

def working_radio(num_years, simulation_size):
    '''
    choose the simulation_size

    use scipy.stats.expon.rvs
    returns
    -------
    probability that the radio will work after 'num_years' years, knowing that
    the number of years a radio functions is exponentially distributed with
    parameter λ = 1 / 8
    '''
    lam = 1/8.
    lifespan = expon.rvs(size=simulation_size, scale = 1/float(lam))
    return len(lifespan[lifespan>num_years])/float(simulation_size)

### QUESTION (Uniform distribution)

Let X be the average number of donuts a data scientist eats per week. X is uniformly distributed from 1/2 to 10 donuts, inclusive.

What is the probability that a randomly selected data scientist eats an average of more than 5 donuts.

#### ANSWER
0.526

#### EXPLANATION
Use cumulative density function for continuous distribution = (x-a)/(b-a)
So, in this case the probability that a data scientist eats 5 donuts or fewer: (5-.5)/(10-.5) = .474

But, we need the complement of this... so 1 - CDF = .526

In [ ]:
  from scipy.stats import uniform

def eating_donut_probability(number_donut, bottom_num, top_num):
    '''
    use uniform.sf (1-cdf)

    return
    ------
    probability that a randomly selected data scientist eats an average of more than 'number_donut' donuts. X, the average number of donut, is uniformly distributed from 'bottom_num' to 'top_num' donuts, inclusive.
    '''
    return uniform.sf(number_donut, loc=bottom_num, scale=top_num-bottom_num)

### QUESTION (Normal distribution)
Suppose that X is a normal random variable with mean 5. If P(X > 9) = .2, approximately what is Var(X )?

#### ANSWER:
22.675

#### EXPLANATION:
```
  If P(X > 9) = .2
  then P(X < 9) = .8
  reverse look up .8 on normal table (i.e., find z score by using a cumulative probability)
  z score of a cumulative probability of .8 = .84
  stddev = (x - mu) / z score
         = (9 - 5)/.84
         = 4.7619
  variance = stddev ^2
           = (4.7619)^2
           = 22.675
```

In [120]:
  from scipy.stats import norm

def get_variance(mean, cutoff, proba_over_cutoff):
    '''
    use scipy.stats.norm.ppf, in inverse of cdf
    returns
    -------
    the variance Var(X) knowing that X is a normal random variable with a
    distribution
      - centered at 'mean',
      - such that P(x > 'cutoff') = 'proba_over_cutoff'
    '''
    return ((cutoff - mean) / norm.ppf(proba_over_cutoff)) ** 2

# Conditional Probability and Bayes Theorem

### 1. Stock prices

A simplified model for the movement of the price of a stock supposes that on each day the stock’s price either moves up 1 unit with probability `p` or moves down 1 unit with probability `1 − p`. The changes on different days are assumed to be independent.

(a) What is the probability that after 2 days the stock will be at its original price?

(b) What is the probability that after 3 days the stock’s price will have increased by 1 unit?

(c) Given that after 3 days the stock’s price has increased by 1 unit, what is the probability that it went up on the first day?

  ### ANSWER: 
                (a) p * (1 - p)
                (b) 2p * (1 - p)
                (c) 2/3

  #### EXPLANATION:
                    (a) union of two events
                    (b) union of three events
                    (c) the price went up twice and down once, and that could
                        have happened on any of the three days



### 2. Getting a new job

A worker has asked her supervisor for a letter of recommendation for a new job. She estimates that there is an 80 percent chance that she will get the job if she receives a strong recommendation, a 40 percent chance if she receives a moderately good recommendation, and a 10 percent chance if she receives a weak recommendation. She further estimates that the probabilities that the recommendation will be strong, moderate, and weak are .7, .2, and .1, respectively.

(a) How certain is she that she will receive the new job offer?

(b) Given that she does receives the offer, how likely should she feel that she received a strong recommendation? a moderate recommendation? a weak recommendation?

(c) Given that she does not receive the job offer, how likely should she feel that she received a strong recommendation? a moderate recommendation? a weak recommendation?

#### ANSWER: 
(a) 0.65
(b) .862, .123, .015
(c) .4, .343, .257


#### EXPLANATION: 
                    (a) .7 * .8 + .2 * .4 + .1 * .1
                        = .56 + .08 + .01
                        = .65
                    (b) Using Bayes' Theorem...
                        p(A|B) = p(B|A) * p(A) / p(B)
                        p(strongrec | job)
                        = p(job | strongrec) * p(strongrec) / p(job)
                        = .8 * .7 / .65
                        = .862

                        = p(job | moderaterec) * p(moderaterec) / p(job)
                        = .4 * .2 / .65
                        = .123

                        = p(job | weakrec) * p(weakrec) / p(job)
                        = .1 * .1 / .65
                        = .015

                    (c) = p(nojob | strongrec) * p(strongrec) / p(nojob)
                        = .2 * .6/ .35
                        = .4

                        = p(nojob | moderaterec) * p(moderaterec) / p(nojob)
                        = .4 * .2/ .35
                        = .343

                        = p(nojob | weakrec) * p(weakrec) / p(nojob)
                        = .9* .1/ .35
                        = .257


### 3. Medical study and some code

A medical study is looking at a test to detect disease that impacts 1 individual in 10. The data collected has shown that
- when a patient has the disease, the test is positive in 90% of the cases
- when a patient does not have the disease, the test is positive in 1% of the cases.

(a) If the test is positive, what is the probability that the patient as that disease?

(b) What if it is a rare disease, that impacts 1 in 10 000 individuals?

(c) How about if the test is negative?

   #### ANSWER: 
               (a) .909
               (b) .009
               (c) .011

  #### EXPLANATION: Using Bayes' Theorem (expanded version):
                    p(A|B) = p(B|A) * p(A) / (p(B|A)* p(A) + p(B|!A) p(!A))

                (a) where A = having disease, B = testing positive, p(A) = .1
                    .9 * .1 / (.9 * .1 + .01 * (1 - .1))
                  = .09 / .009
                  = .909

                (b) where p(A) = 1/10000
                    .9 * .0001 / (.9 * .0001 + .01 * (1-.0001)
                    = .00892

                (c) where B = testing negative
                    (1 - .9) * .1 / ((1 - .9) * .1 + (1 - .01)* (1 -.1))
                    .011


In [123]:
def positive_test(TP, FP, perc_population):
    '''
    parameters
    ----------
    TP: true positive
      percentage of tests that were positive
      for the sample of subjects that had the disease
    FP: false positive
      percentage of tests that were positive
      for the control population (disease-free subjects)

    percent_population: percentage of the population that has the disease

    returns
    -------
    probability of having the disease for a person with a positive test result
    '''
    return TP * perc_population / \
        (TP * perc_population + FP *(1-perc_population))